In [1]:
import cv2 
import numpy as np
import os
import numpy as np
from datetime import datetime

In [2]:
output_folder = 'C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/HappyScenario'
os.makedirs(output_folder, exist_ok=True)
print(f'output_folder: {output_folder}')

output_folder: C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/HappyScenario


In [3]:
img = cv2.imread('data\sample.jpg')
img_gray = cv2.imread('data\sample.jpg', cv2.IMREAD_GRAYSCALE)
template = cv2.imread(r'data\test.jpg', cv2.IMREAD_GRAYSCALE)

print(f'image shape: {img_gray.shape}')
print(f'template shape: {template.shape}')

image shape: (9000, 12600)
template shape: (61, 48)


In [7]:
# Load images
image = cv2.imread('data\sample.jpg')
templates = ['data\exm\disconnectswitch.jpg', 'data\exm\duplexrece.jpg']

# Convert to grayscale
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Initialize SIFT or ORB detector
sift = cv2.SIFT_create()
# orb = cv2.ORB_create()

# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

In [8]:
for template_path in templates:
    # Load template and convert to grayscale
    template = cv2.imread(template_path)
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    # Find key points and descriptors
    kp1, des1 = sift.detectAndCompute(template_gray, None)
    kp2, des2 = sift.detectAndCompute(image_gray, None)

    # Match descriptors using FLANN
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Store good matches based on distance ratio
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    # Draw matches if sufficient good matches are found
    if len(good_matches) > 10:  # Adjust threshold as needed
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Find homography if enough matches are found
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)
        h, w = template_gray.shape
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, M)

        # Draw bounding box around detected template
        image = cv2.polylines(image, [np.int32(dst)], True, (0, 255, 0), 3)

# Create a filename based on scale and threshold
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"detected_image_{current_time}.jpg"
filepath = os.path.join(output_folder, filename)

# Save the detected image
cv2.imwrite(filepath, image)

# Show result
cv2.imshow('Detected Templates', image)
cv2.waitKey(0)
cv2.destroyAllWindows()